In [ ]:
import numpy as np
from numpy import random as rand

import matplotlib
from matplotlib import pyplot as pt
%matplotlib inline

In [ ]:
N = 15

def init_lattice():
    lattice = np.random.choice([-1,1], size = (N,N))
    return lattice


x = init_lattice()
x

In [ ]:
def nearest_neighbours(lattice, x, y):
    
    # assume the lattice is isotropic, with matching boundary conditions
    # the nearest neighbpurs do not include the diagonals
    
    neighbour_spin = []
    
    if  0 <= (y-1) < N : 
        left   = (x, (y-1))
        neighbour_spin.append(lattice[left[0], left[1]])
                                   
    if 0 <= (y+1) < N : 
        right  = (x, (y+1))
        neighbour_spin.append(lattice[right[0], right[1]])
        
    if 0 <= (x-1) < N :
        top    = ((x-1), y)
        neighbour_spin.append(lattice[top[0], top[1]])
    
    if 0<= (x+1) < N :
        bottom = ((x+1), y)
        neighbour_spin.append(lattice[bottom[0], bottom[1]])
    
    # create an array containing spins of the nearest neighbours 
    # order turns out to not be important, only the total sum
    
    return neighbour_spin

nearest_neighbours(x,9,9)

In [ ]:
def sum_spins(lattice, x, y):

    sum_spins = sum(nearest_neighbours(lattice,x,y))
    return sum_spins
    
sum_spins(x,2,3)

In [ ]:
def del_energy_local(lattice, x, y):
    # we note the change in energy of the lattice if a single local spin is reverse
    # stablility is proportional to the number of nearest neighbours (markov chain assumption) with the same spin 
    # stability = lower energy
    # this is not the actual change in energy but it id proportional to it and thus this model is useful
    
    dE = -1 * lattice[x,y] * sum_spins(lattice, x, y)
    return dE

del_energy_local(x,0,4)

In [ ]:
def del_energy_total(lattice):
    
    # work through all nodes to get the total change
    DE = 0
    
    for i in range(0, N):
        for j in range(0,N):
            DE += del_energy_local(lattice, i, j)
            
    return DE

y = init_lattice()
del_energy_total(x)

In [ ]:
import random

def monte_carlo(lattice,temp):
        
        #random coordinates
        x = random.randint(0,N-1)
        y = random.randint(0,N-1)
        
        cost = del_energy_local(lattice,x,y)
        
        
        
        # clearly reduces energy
        if cost < 0:
            
            lattice[x,y] = -1 * lattice[x,y]
        # does not reduce energy, compare to uniform distribution
        
        elif np.exp((-1.0 * (cost - 1.8) )/ temp) < rand.random():
            
             lattice[x,y] = -1 * lattice[x,y]
                
                
        return lattice 

In [ ]:
temp_ticks = 40      
iterations = 500
eq_iterations = 500

T = np.linspace(1.0,3.5,temp_ticks)
E = np.zeros(temp_ticks)
M = np.zeros(temp_ticks)

k = 1.0/(iterations * N * N)

In [ ]:
def metropolis_alg(lattice):
    
    for i in range(0,temp_ticks):
        
        temp = T[0] + ((T[temp_ticks-1]-T[0])*i)/temp_ticks    
        
        a = monte_carlo(lattice,temp)
        
        for j in range(0,iterations):
        
            a = monte_carlo(a,temp) 
           
        if (i == temp_ticks/2) and (j % 256 == 0):
            pt.imshow(a, cmap='hot', interpolation='nearest')
            pt.show()
    
    E[i] = del_energy_total(a)
        
    return E   


In [ ]:
def average_equilibriate(lattice):
    
    for i in range(0,temp_ticks):
        
        temp = T[0] + ((T[temp_ticks-1]-T[0])*i)/temp_ticks
        E_1 = 0
        a = lattice
        E_0 = del_energy_total(a)
        
        for j in range(eq_iterations):            #equilibriate
            a = monte_carlo(a, temp)
        
        #for k in range(iterations):
        #   a = monte_carlo(a, temp)           #average energy
            energy = del_energy_total(a)
        
            E_1 = E_1 + energy
        
        E[i] = (E_1/ (500)) - E_0
    
    return E

In [ ]:
def monte_carlo_stepwise(lattice,temp):
        
        #random coordinates
    for i in range(0,N):
        for j in range(0,N):
        
            cost = del_energy_local(lattice,i,j)
        
        
        
            # clearly reduces energy
            if cost < 0:
            
                lattice[i,j] = -1 * lattice[i,j]
            # does not reduce energy, compare to uniform distribution
        
            elif np.exp((-1.0 * (cost - 1.8))/ temp) < rand.random():
            
                lattice[i,j] = -1 * lattice[i,j]
                
                
    return lattice 

In [ ]:
def average_stepwise(lattice):
    
    for i in range(0,temp_ticks):
        
        temp = T[0] + ((T[temp_ticks-1]-T[0])*i)/temp_ticks
        E_0 = del_energy_total(lattice)
        
       # a = lattice
        #for j in range(15):            
        a = monte_carlo_stepwise(lattice, temp)
                #for k in range(iterations):
        #   a = monte_carlo(a, temp)           
        energy = del_energy_total(a)
        
            
        
        E[i] = energy - E_0
    
    return E